In [ ]:
import sys
lib_path = [r'C:\Users\ikahbasi\OneDrive\Applications\GitHub\SeisRoutine',
            r'C:\Users\ikahb\OneDrive\Applications\GitHub\SeisRoutine']
for path in lib_path:
    sys.path.append(path)
##########################################################################
import SeisRoutine.catalog as src
import SeisRoutine.waveform as srw
import SeisRoutine.config as srconf
import SeisRoutine.statistics as srs

In [ ]:
import seisbench.data as sbd
import seisbench.generate as sbg
import numpy as np
import os
from scipy import signal
from tqdm import tqdm
from scipy.stats import skew

In [ ]:
init_cfg = srconf.load_config('0-init-cfg.yml')
cfg_path = os.path.join(init_cfg.target_config_filepath,
                        init_cfg.target_config_filename)
cfg = srconf.load_config(cfg_path)

In [ ]:
dataset = sbd.WaveformDataset(
    path=cfg.dataset.path,
    sampling_rate=cfg.training.dataset.sampling_rate,
    component_order=cfg.training.dataset.component_order,
          )

In [ ]:
sps = 100
augmentations = [
    sbg.Normalize(demean_axis=-1,
                  amp_norm_axis=-1,
                  amp_norm_type="peak"),
    sbg.ChangeDtype(np.float32),
]
generator = sbg.GenericGenerator(dataset)
generator.add_augmentations(augmentations)

In [ ]:
metadata = dataset.metadata.copy()
for ii in tqdm(range(len(metadata))):
    data = generator[ii]
    data_X = data['X']
    std = data_X.std(axis=1)
    for channel, std_channel in zip(dataset.component_order, std):
        metadata.at[ii, f'trace_{channel}_std'] = std_channel

In [ ]:
# list(dataset.metadata.keys())
keys = ['trace_start_time', 'trace_channel',
        'station_code', 'station_network_code',
        'source_origin_time', 'trace_name',
        'source_id'] + [key for key in metadata.keys()
                        if key.endswith('_std')]

In [ ]:
path = os.path.join(cfg.dataset.path, 'metadata-with-std.pkl')
metadata[keys].to_pickle(path)